In [33]:
import pandas as pd
from mitreattack.stix20 import MitreAttackData


mitre_attack_data = MitreAttackData("enterprise-attack.json")

# cve to cwe and severity matrix
cve_cwe_df = pd.read_excel('cve_to_cwe.xlsx')

# cve to technique mapping
cve_df  = pd.read_csv('cve_mapping.csv')

# nist mapping
nist_df = pd.read_csv('nist_800_53_mapping.csv')
# cve_df[cve_df['mapping_type'].str.contains('primary_impact')]
#cve_df.columns

# technique_using_groups = mitre_attack_data.get_all_techniques_used_by_all_groups()
groups_list = []


for id, technique  in technique_using_groups.items():
    group_id = mitre_attack_data.get_attack_id(id)
    # print(f"{group_id} - uses  {len(technique)} techniques")
    mitre_id_list = []

    for t in technique:
        external_id = t['object'].external_references[0].external_id 
        mitre_id_list.append(external_id)
    groups_list.append([group_id,mitre_id_list])

cve_df.rename(columns={"capability_id": "CVE-ID"}, inplace=True)

for group_id, mitre_list in groups_list:
    print(f"{group_id} group uses")
    cves = get_cves_used_by_techniques(mitre_list)
    print(cves)
    # newdf = pd.merge(cves, cve_cwe_df, how='inner', on=['CVE-ID'])
    # print(newdf['CVSS-V2'].mean(), end=' - ')
    # print(newdf['CVSS-V3'].mean(), end=' - capability counts - ')
    # filt = nist_df['attack_object_id'] == t
    # display(nist_df.loc[filt]['capability_group'].value_counts())
    # display(newdf)
    break
# nist_df['capability_group'].value_counts()




G0119 group uses
{'CVE-2018-15782', 'CVE-2020-11082', 'CVE-2014-4148', 'CVE-2020-5339', 'CVE-2014-0295', 'CVE-2015-1642', 'CVE-2010-1424', 'CVE-2014-6352', 'CVE-2020-3233', 'CVE-2020-0671', 'CVE-2009-3129', 'CVE-2020-11030', 'CVE-2020-3349', 'CVE-2019-0609', 'CVE-2020-11023', 'CVE-2014-0266', 'CVE-2020-4068', 'CVE-2020-11010', 'CVE-2018-11073', 'CVE-2019-13541', 'CVE-2019-1915', 'CVE-2019-1035', 'CVE-2019-3754', 'CVE-2018-8248', 'CVE-2015-0096', 'CVE-2020-1569', 'CVE-2015-6475', 'CVE-2020-5270', 'CVE-2019-0576', 'CVE-2019-18571', 'CVE-2020-11073', 'CVE-2019-15821', 'CVE-2020-5336', 'CVE-2020-11901', 'CVE-2019-1620', 'CVE-2010-0028', 'CVE-2020-5340', 'CVE-2020-5271', 'CVE-2010-3333', 'CVE-2019-0911', 'CVE-2014-0810', 'CVE-2019-13538', 'CVE-2019-3719', 'CVE-2009-3459', 'CVE-2020-3126', 'CVE-2014-4123', 'CVE-2020-1111', 'CVE-2019-1118', 'CVE-2019-3798', 'CVE-2020-5231', 'CVE-2018-8111', 'CVE-2019-15280', 'CVE-2018-15376', 'CVE-2019-13527', 'CVE-2020-5269', 'CVE-2017-11882', 'CVE-2020-1103

In [32]:
# given a list of techniques [] return all the cves related to the techniques
# it uses a set to eleminate duplicates for a set
def get_cves_used_by_techniques(techniques):
    cvs = set()
    for t in techniques:
        cvs.update(cve_df[cve_df['attack_object_id'].str.contains(t)]['CVE-ID'].to_list())
    return cvs
        
    

In [ ]:



# technique = mitre_attack_data.get_object_by_attack_id('T1134', 'attack-pattern')

# groups_using_techniques = mitre_attack_data.get_all_groups_using_all_techniques()

# print(technique)
# print(f"Groups using techniques ({len(groups_using_techniques.keys())} techniques):")

# for id, groups in groups_using_techniques.items():
#     attack_id = mitre_attack_data.get_attack_id(id)
#     print(f"{attack_id} - used by {len(groups)} {'group' if len(groups) == 1 else 'groups'}")

# print(f"Technics used by ({len(technique_using_groups.keys())} groups):")

# print(technique_using_groups.items()[0])
# print(technique_using_groups.items())
    # for m in mitre_id_list:
    #     print(m)




